In [1]:
import arcpy
arcpy.env.overwriteOutput = True 
from arcpy.sa import *
from arcpy import env

import os
import pandas as pd
import numpy as np


workspace = os.path.join("..", 'RESULTS')
#make sure workspace directory exists
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
tempspace = os.path.join(workspace, "temp")
#make sure workspace directory exists
if not os.path.exists(tempspace):
    os.makedirs(tempspace)


outspace = os.path.join(workspace, 'Outputs')
#make sure workspace directory exists
if not os.path.exists(outspace):
    os.makedirs(outspace)

    
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container { width: 95%;} div#menubar-container { width: 85%; } div#maintoolbar-container { width: 99%; } </style> """))

OK so gotta refactor to find a better way to extract all data from each scenario and compile into one frame. There will be
- Base Flooded 0m
- Base Flooded 1 m

- SLR 2ft flooded 0m reg recharge
- SLR 2ft flooded 1m reg recharge
- SLR 2ft flooded 0m RCP4.5 recharge
- SLR 2ft flooded 1m RCP4.5 recharge

- SLR 5ft flooded 0m reg recharge
- SLR 5ft flooded 1m reg recharge
- SLR 5ft flooded 0m RCP4.5 recharge
- SLR 5ft flooded 1m RCP4.5 recharge

- SLR 8ft flooded 0m reg recharge
- SLR 8ft flooded 1m reg recharge
- SLR 8ft flooded 0m RCP4.5 recharge
- SLR 8ft flooded 1m RCP4.5 recharge



SLR_coastline file from NOAA Processing Wrokflow
- simplify polygon on file 5 m  = Simplified
- run a union to remove the gaps, make sure gaps are not allowed manually edit to merge all polys together = gap_union
- run a union with the model square to get the land area as polygon  = square_union.shp
- at this point export square_uniin with no dumb fields make OceanCell fields 1 for land -1 fpr ocean
- union again on the 100 m buffer square for heads Model_square_100mBuf.shp  = buff_union  (Make a backup if mess up on editing)
- make ocean cells ibound field -1 and land 1
- Export as ibound_specHd_w_100m_buff_SLRXft

In [2]:
#ascs
DtW_base = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5p2', "dep2Wat__base.asc")
DtW_8ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR8ft', "dep2Wat_SLR8ft.asc")
DtW_5ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR5ft', "dep2Wat_SLR5ft.asc")
DtW_2ft = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR2ft', "dep2Wat_SLR2ft.asc")

DtW_8ft_RCP45 = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR8ft', "dep2Wat_SLR8ft_RCP45.asc")
DtW_5ft_RCP45 = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR5ft', "dep2Wat_SLR5ft_RCP45.asc")
DtW_2ft_RCP45 = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files/Flo_py_wrkspace5_SLR2ft', "dep2Wat_SLR2ft_RCP45.asc")

#  path to the infrastructure layers 
InfraShpPath = os.path.join("..", 'Data/Raw/GIS/Infrastructure/Clipped')


Depths_to_clip_infrastructure = {"Surface": 0.01,  "Sub": 1.01}

SLR_scenarios = ["base", "2ft", "5ft", "8ft", "2ft_RCP45", "5ft_RCP45", "8ft_RCP45"]    # Each is a raster in a separate folder

Scenaio_raster_dic = {
    "base":        DtW_base, 
    "2ft":         DtW_2ft,   
    "2ft_RCP45":   DtW_2ft_RCP45,    
    "5ft":         DtW_5ft,    
    "5ft_RCP45":   DtW_5ft_RCP45,    
    "8ft":         DtW_8ft,   
    "8ft_RCP45":   DtW_8ft_RCP45}

Polyline_infrstructures = ['Relevant_Sewer_Lines_core.shp', 'ASPA_water_Distribution_core.shp', 
                           'ASPA_water_Transmission_core.shp', 'Roads_2006_core.shp'] 

Point_infrstructures = ['ASPA_CustomerMeters_core.shp', 'ASPA_water_Boosters_core.shp', 'All_sewer_LiftStations_core.shp'] 

In [3]:
# Create shapefiles and a dic of the total lengths of polyline files 

OG_lenth_dic = {}
for i in Polyline_infrstructures:
    
    shapefile_to_measure = os.path.join(InfraShpPath, i)
    
    # First copy the original features to calculate and print the total lengths of polylines later
    arcpy.management.CopyFeatures(shapefile_to_measure, os.path.join(tempspace, "Length_{}".format(i)))

    fc_OG = os.path.join(tempspace, "Length_{}".format(i))
    arcpy.AddField_management(fc_OG, 'lenOG_m','DOUBLE')
    arcpy.management.CalculateField(fc_OG, "lenOG_m", "!SHAPE.geodesicLength@METERS!")   
    field = arcpy.da.TableToNumPyArray (fc_OG, "lenOG_m", skip_nulls=True)
    tot_len = field["lenOG_m"].sum()
    print("Total Original Length of {} is {} m".format(i, tot_len)) 
    OG_lenth_dic[i] = tot_len
    
    
for i in Point_infrstructures:
    fc = os.path.join(InfraShpPath, i)
    field = arcpy.da.TableToNumPyArray (fc, "FID", skip_nulls=True)
    tot_things = len(field)
    OG_lenth_dic[i] = tot_things
    print("Total number of {} is {}".format(i, tot_things))
    

# Turn the Dic into a dataframe
OG_lenth = pd.DataFrame.from_dict([OG_lenth_dic], orient='columns').transpose()
OG_lenth = OG_lenth.rename(columns={0:"Original_length_or_number"})
OG_lenth

Total Original Length of Relevant_Sewer_Lines_core.shp is 42315.5045899074 m
Total Original Length of ASPA_water_Distribution_core.shp is 44057.93550498824 m
Total Original Length of ASPA_water_Transmission_core.shp is 47137.500621334475 m
Total Original Length of Roads_2006_core.shp is 76765.56681189116 m
Total number of ASPA_CustomerMeters_core.shp is 1329
Total number of ASPA_water_Boosters_core.shp is 5
Total number of All_sewer_LiftStations_core.shp is 8


,Original_length_or_number
ASPA_CustomerMeters_core.shp,1329.000000
ASPA_water_Boosters_core.shp,5.000000
ASPA_water_Distribution_core.shp,44057.935505
ASPA_water_Transmission_core.shp,47137.500621
All_sewer_LiftStations_core.shp,8.000000
Relevant_Sewer_Lines_core.shp,42315.504590
Roads_2006_core.shp,76765.566812


### Main cell: Assess flooded infrastructure

In [9]:

for depth in Depths_to_clip_infrastructure:
    
    for scenario in SLR_scenarios:
        
        # Create polygons for Surface or subsurface "flooded" areas for each of the scenarios in SLR_scenarios:
        in_raster = Scenaio_raster_dic[scenario]
        outraster = os.path.join(tempspace, 'Temp_raster.tif')
        DaNewraster = Raster(in_raster) < Depths_to_clip_infrastructure[depth]
        DaNewraster.save(outraster)

        # Convert the raster to polygons
        outPolygons = os.path.join(tempspace, 'temp.shp')
        arcpy.RasterToPolygon_conversion(outraster, outPolygons, "NO_SIMPLIFY")

        ## Delete any areas where the raster (now gridcode attribute) is 0
        arcpy.MakeFeatureLayer_management(outPolygons, "Temp_Poly")
        Selection = arcpy.SelectLayerByAttribute_management("Temp_Poly", "NEW_SELECTION", "gridcode > 0")
        arcpy.CopyFeatures_management(Selection, os.path.join(tempspace, 'Flooded_Area_{}_{}.shp'.format(scenario, depth)))
            
        results_dic = {}
        for file in os.listdir(InfraShpPath):
            if file.endswith(".shp"):
                shapefile_to_clip = os.path.join(InfraShpPath, file) 

                # Now do the actual clipping of flooded areas
                out_clipped  = os.path.join(outspace, "{}_{}_Flooded_{}".format(scenario, depth, file))
                arcpy.Clip_analysis(shapefile_to_clip, os.path.join(tempspace, 'Flooded_Area_{}_{}.shp'.format(scenario, depth)), out_clipped) 
                
                
                # NOW work on the Polyline_infrstructures:                                 
                if file in Polyline_infrstructures:
                    arcpy.AddField_management(out_clipped, 'lenFlod_m','DOUBLE')
                    arcpy.management.CalculateField(out_clipped, "lenFlod_m", "!SHAPE.geodesicLength@METERS!")   
                    field = arcpy.da.TableToNumPyArray (out_clipped, "lenFlod_m", skip_nulls=True)
                    flod_len = field["lenFlod_m"].sum()
                    print("Length of flooded {} is {} m in {} at {} depth".format(file, flod_len, scenario, depth)) 
                    
                    pct_flooded = round((flod_len/OG_lenth_dic[file])*100, 1)
                    print("Total length of {} is {}, Percent of flooded is {}%  \n".format(file, OG_lenth_dic[file], pct_flooded))
                   
                    results_dic[file] = flod_len
                    
                                      
               # NOW work on the point_infrstructures: 
                if file in Point_infrstructures:
                    field = arcpy.da.TableToNumPyArray (out_clipped, "FID", skip_nulls=True)
                    flood_things = len(field)
                    print("Number of flooded {} is {}".format(i, flood_things))
                    
                    pct_flooded = round((flood_things/OG_lenth_dic[file])*100, 1)
                    print("Total number of {} is {}, Percent of flooded is {}%  \n".format(file, OG_lenth_dic[file], pct_flooded))
                
                    results_dic[file] = flood_things     
        
        Flooded_scenarios_dic[scenario+"_"+depth] = results_dic
                

Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of All_sewer_LiftStations_core.shp is 8, Percent of flooded is 0.0%  

Number of flooded All_sewer_LiftStations_core.shp is 1
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 0.1%  

Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 0.0%  

Length of flooded ASPA_water_Distribution_core.shp is 160.01415771860002 m in base at Surface depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 0.4%  

Length of flooded ASPA_water_Transmission_core.shp is 60.012337816969996 m in base at Surface depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 0.1%  

Length of flooded Relevant_Sewer_Lines_core.shp is 53.33366692783 m in base at Surface depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 0

Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of All_sewer_LiftStations_core.shp is 8, Percent of flooded is 0.0%  

Number of flooded All_sewer_LiftStations_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 1.6%  

Number of flooded All_sewer_LiftStations_core.shp is 1
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 20.0%  

Length of flooded ASPA_water_Distribution_core.shp is 1627.9228571578751 m in base at Sub depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 3.7%  

Length of flooded ASPA_water_Transmission_core.shp is 1853.3616472042809 m in base at Sub depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 3.9%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1461.7971693460172 m in base at Sub depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 3.5%  


In [43]:
# Post processing and syntehesis 

Flooded_scenarios_dic["Total_Lengths"] = OG_lenth_dic     # Add in the total lengths/numbers

output = pd.DataFrame.from_dict(Flooded_scenarios_dic, orient='columns')   # make it all a datframe
output = output.transpose()   # Transpose it

# Create a dataframe of percentage flooded
output_pct = output.copy()
for col in output: 
    tot_things = output[col].loc['Total_Lengths']
    output_pct[col] = output_pct[col]/tot_things
    
# Format percentages 
output_pct['sum']  = output_pct.sum(axis=1)/7
output_pct = output_pct.round(3)*100

# Sort by best to worst scenario 
output_pct = output_pct.sort_values("sum")

# Sort the in m df by best to worst scenario  
output['tot_buried_lines'] = output[['ASPA_water_Distribution_core.shp', 'ASPA_water_Transmission_core.shp', 'Relevant_Sewer_Lines_core.shp']].sum(axis=1)
output = output.sort_values('tot_buried_lines')

#Save csvs
output_pct.to_csv(os.path.join("..", 'RESULTS', 'output_pct.csv'))
output.to_csv(os.path.join("..", 'RESULTS', 'output.csv'))

output_pct

,ASPA_CustomerMeters_core.shp,ASPA_water_Boosters_core.shp,ASPA_water_Distribution_core.shp,ASPA_water_Transmission_core.shp,All_sewer_LiftStations_core.shp,Relevant_Sewer_Lines_core.shp,Roads_2006_core.shp,sum
base_Surface,0.1,0.0,0.4,0.1,0.0,0.1,0.1,0.1
2ft_Surface,0.1,0.0,0.6,0.5,0.0,0.3,0.4,0.3
2ft_RCP45_Surface,0.2,0.0,0.7,0.7,0.0,0.6,0.5,0.4
base_Sub,1.6,20.0,3.7,3.9,0.0,3.5,3.8,5.2
5ft_Surface,2.1,20.0,4.3,5.9,0.0,4.1,4.5,5.8
5ft_RCP45_Surface,2.1,20.0,4.3,6.1,0.0,4.2,4.7,5.9
2ft_Sub,4.9,20.0,9.0,13.8,0.0,11.2,13.0,10.3
2ft_RCP45_Sub,5.3,20.0,9.5,14.9,0.0,12.1,14.2,10.8
8ft_RCP45_Surface,10.7,20.0,17.1,32.8,37.5,23.3,28.0,24.2
8ft_Surface,10.7,20.0,17.1,32.8,37.5,23.3,28.0,24.2


### Some context and results notes

TO consider: 
- should report results of surface inundated features (DtW_level of 0) and subsurface inundated features (DtW_level of 1) too


The calibrated base scenario seems to do a reasonably good job of showing where average condition groundwater inundation occurrs. Areas in the present day base scenario where water reaches the ground surface match well with areas where there are canals or other persistant nearshore water features (Figure if need) 


Sewer lines includes all force and gravity mains on the asbuilts which we recognize to be an incomplete dataset, but is the best available provided by the utility. 

In present day conditions, of the 9.11 km total of water mains (44.1 km of distribution, 47.1 km of transmission), if we assume a 1 m depth then a total of 3.4 km are consistently inundated with water = 3.7%
In present day conditions, of the 42.3 km total of sewer lines, if we assume a 1 m depth then a total of 1.4 km are consistently inundated with water = 3.3%  
[STILL NEED BURIED ELECTRICAL LINES] 



